<a href="https://colab.research.google.com/github/kainzc/BigData-1/blob/main/Assignment_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment we are going to work with data that was downloaded using using the [yfinance](https://pypi.org/project/yfinance/) library. It containst daily closing prices of all symbols.

In [1]:
from multiprocessing   import Pool
from tqdm import *
import pandas as pd
import datetime

In [2]:
#Question 1: Explain what the code below is trying to do. Describe each step.
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())


In [ ]:
#Describe your answer below.
#Answer 1: 
# The first line is reading in data from a csv file. This csv file contains
# stock information on various companies.

# The second line is filtering rows in 'data' into data_clean based on whether
# or not each row's column value for 'Test Issue' = 'N'.

# Line 3 is assigning 'symbols' equal to the set of all unique values under
# the 'NASDAQ Symbol' column. 

# Line 4 is reading in data from a csv file. This csv file contains
# more data on stocks. It is also updating the data variable.

# Line 5 is filtering rows in 'data' into data_clean based on whether
# or not each row's column value for 'Test Issue' = 'N'.

# Line 6 is assigning 'symbols' equal to the set of all unique values under
# the 'NASDAQ Symbol' column. 

In [3]:
!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data

--2023-01-30 01:48:36--  https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.6.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.6.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: /s/dl/gzpi950gfdz9dsk/stocks.parquet [following]
--2023-01-30 01:48:37--  https://www.dropbox.com/s/dl/gzpi950gfdz9dsk/stocks.parquet
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc16a94bb1ab6984b108f774bbf2.dl.dropboxusercontent.com/cd/0/get/B1icDk8WyovWNW2piavagMEjfStYfICq9JBXEeEEOkw22HXmerktq8N1XoB8GmG92xscGezHtPvoThSDgkZ4tEttlTO_Lnxgoyd6J4aULBL63OTl3D8VRigRvCEVfSFacPJflUb30zYZ8eHBkRDY3dRTgvYgEXdNKydndTFdw53VUw/file?dl=1# [following]
--2023-01-30 01:48:37--  https://uc16a94bb1ab6984b108f774bbf2.dl.dropboxusercontent.com/cd/0/get/B1icDk8WyovWNW2piavagMEjfStYfICq9JBXEeEEOkw22HXmerktq8N1XoB8GmG92xscGezH

In [4]:
!ls stocks.parquet -l #downloaded file

-rw-r--r-- 1 root root 678640589 Jan 30 01:48 stocks.parquet


In [ ]:
#Bonus question 1 - find on internet and write the command that will download the all data from the symbols list. 

#for symbol in symbols:
#    data = pdr.get_data_yahoo(symbol)

# could use a different data structure if you want to hold it all together.
# pdr.get_data_yahoo works for each individual symbol.

In [5]:
data=pd.read_parquet('stocks.parquet')

In [ ]:
data.head(1) # Below shows a sample of the data.
#date - describes the day of the price
#open - the price at which stock opened
#high - what was the intraday high
#low - what is the intraday low
#close - what was the closing price
#adj_close - the price calculated after adjusting for all future stock splits
#dividends - what are the dividends that were paid on the day.
#stock_splits - if the stock was split
#symbol the symbol of the stock



,date,open,high,low,close,adj_close,volume,dividends,stock_splits,symbol,day,year,month,dayofweek
0,1962-01-02,0.131273,0.131273,0.124177,0.124177,0.045278,2480333.0,0.0,0.0,HPQ,2,1962,1,Tuesday


In [16]:
import numpy as np

# Question 2 find top 20 best performer stocks - i.e. the stocks that provided the best return.
# if the stocks adj_close on the earliest date was X and the adj_close on the 2021-01-01 is y and the number of days elapsed between these dates is Z.
# Then return = Y/X/Z
# Sort the stocks by this return and find the top 20

df = data.groupby('symbol')['date', 'adj_close'].agg(['first', 'last'])

df[('date', 'first')] = pd.to_datetime(df[('date', 'first')])
df[('date', 'last')] = pd.to_datetime(df[('date', 'last')])

x, y = df['adj_close', 'first'], df['adj_close', 'last']
z = (df['date']['last'] - df['date']['first']).dt.days




result = y/x/z
result = result.rename('returns').to_frame()
result.replace([np.inf, -np.inf], np.nan, inplace=True) 
result.dropna(inplace=True)
result.sort_values(by='returns', ascending=False, inplace=True)
result = result.head(20)
print(result)
  



<ipython-input-16-9eb2ce1813d4>:8: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  df = data.groupby('symbol')['date', 'adj_close'].agg(['first', 'last'])


             returns
symbol              
NVO     3.302876e+06
NKE     2.959418e+03
DCI     3.869564e+02
TJX     2.063167e+02
WTRG    7.221605e+00
TTC     6.224708e+00
USB     4.078555e+00
MCD     3.374097e+00
PII     2.554223e+00
HD      1.506773e+00
IQMD    1.001014e+00
AOGOU   9.990020e-01
PCCT    9.959839e-01
ETN     9.076372e-01
FRLAW   8.888889e-01
HAS     7.035260e-01
KO      6.680578e-01
MO      6.485381e-01
KOL     5.786728e-01
HRL     5.406652e-01


In [ ]:
#Question 3: Create a copy of this notebook called assignment-aws and follow the instructions in readme to run the notebook on AWS compute instance.
# Compare performance of running natively on colab and colab+AWS